In [1]:
import numpy as np
import pandas as pd
import vartools as vt

## Ejercicio 1

In [2]:
df = pd.read_excel("18.- Repaso_2_EX_P2025.xlsx", sheet_name=1)
df = df.set_index('Date')
posiciones1 = np.array([121*100, 153*5000, 189*25000])
conf = 0.99
long = False

In [3]:
vt.var_apl(df, posiciones1, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0244,"1,469,942.9064"
1,VaR Ajustado Promedio,0.0254,"1,529,102.2907"
2,VaR Ajustado Estresado,0.0289,"1,740,885.0820"
3,C-VaR,0.0289,"1,738,382.5541"
4,C-VaR Ajustado Promedio,0.0299,"1,797,541.9383"
5,C-VaR Ajustado Estresado,0.0334,"2,009,324.7297"


In [4]:
adv = df['Volume'].mean()
n_shares = 121*100

In [5]:
np.ceil(n_shares/adv)

1.0

## Ejercicio 2

In [6]:
data = pd.read_excel("18.- Repaso_2_EX_P2025.xlsx", sheet_name=3)
data = data.set_index('Date')
posiciones = np.array([1_571_000, 7_203_000])
conf = 0.99
long = True

In [7]:
vt.var_apl(data, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0147,"46,302.2918"
1,VaR Ajustado Promedio,0.0150,"47,136.1542"
2,VaR Ajustado Estresado,0.0153,"48,068.5437"
3,C-VaR,0.0171,"53,853.4672"
4,C-VaR Ajustado Promedio,0.0174,"54,687.3296"
5,C-VaR Ajustado Estresado,0.0177,"55,619.7191"


In [8]:
def var_apl(data: pd.DataFrame, posiciones: list | np.ndarray, conf: float, long: bool):
    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost

    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return np.dot(value, cl_prom)

In [9]:
var_apl(data, posiciones, conf, long)

833.8623314070503